In [90]:
import pandas as pd
import os
from matplotlib import pyplot as plt

def combine_files_by_base_name(directory):
    """
    Combines files in a directory based on their base name, excluding certain files.

    Parameters:
        directory (str): Path to the directory containing files to combine.

    Returns:
        dict: A dictionary where keys are base names and values are combined DataFrames.
    """
    file_groups = {}

    for filename in os.listdir(directory):
        if "gpt" not in filename:
            continue
        base_name = '_'.join(filename.split('_part')[0].split('_')[:-1])
        if base_name not in file_groups:
            file_groups[base_name] = []
        file_groups[base_name].append(filename)

    combined_dfs = {}
    for base_name, files in file_groups.items():
        combined_df = pd.DataFrame()

        for filename in sorted(files):  # Ensure files are processed in order of parts
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)

        combined_dfs[base_name] = combined_df

    return combined_dfs

# Helper function to separate attacks based on step_num reset
def separate_attacks(df, length=10000):
    attacks = []
    current_attack = []
    
    for idx, row in df.iterrows():
        # Start a new attack if the step_num resets
        if idx > 0 and (row['Num'] != df.loc[idx - 1, 'Num'] + 1 or row['Origin'] != df.loc[idx - 1, 'Origin']):
            attacks.append(pd.DataFrame(current_attack))
            current_attack = []        

        current_attack.append(row)
    
    # Append the last attack
    if current_attack:
        attacks.append(pd.DataFrame(current_attack))
    
    return attacks

def add_entropy_column(dfs):
    """
    Takes a list of DataFrames, checks if the length of the list is a multiple of 10,
    and adds an 'entropy' column to each DataFrame, containing the remainder modulo 10 of the DataFrame's position in the list.

    Parameters:
        dfs (list of pd.DataFrame): List of DataFrames.

    Returns:
        list of pd.DataFrame: The updated list of DataFrames with the added 'entropy' column.

    Raises:
        ValueError: If the length of the list is not a multiple of 10.
    """
    if len(dfs) % 10 != 0:
        raise ValueError("The length of the list must be a multiple of 10.")

    for i, df in enumerate(dfs):
        df['entropy'] = i % 10

    return dfs

def process_and_add_entropy(file_path, separate_attacks, add_entropy_column):
    """
    Streamlines the processing of a CSV file: reads it, extracts domain from filename,
    adds a domain column, separates attacks, and adds an entropy column.

    Parameters:
        file_path (str): Path to the CSV file.
        separate_attacks (function): Function to separate attacks from the DataFrame.
        add_entropy_column (function): Function to add an entropy column to the list of DataFrames.

    Returns:
        list of pd.DataFrame: List of processed DataFrames with entropy column added.
    """
    domain = os.path.basename(file_path).split('_')[-2]
    df = pd.read_csv(file_path)
    df['domain'] = domain
    attacks = separate_attacks(df)
    return add_entropy_column(attacks)

def check_unique_values(df, columns):
    """
    Checks whether specified columns in a DataFrame have a single unique value and extracts this value.

    Parameters:
        df (pd.DataFrame): The DataFrame to check.
        columns (list of str): List of column names to check.

    Returns:
        dict: A dictionary with column names as keys and their unique value if single, else None.
    """
    unique_values = {}
    for col in columns:
        unique_vals = df[col].unique()
        if len(unique_vals) == 1:
            unique_values[col] = unique_vals[0]
        else:
            unique_values[col] = None
    return unique_values

def return_correct_flipped_correct(df, prefix=""):
    df.loc[:, f'{prefix}correct'] = (df['Origin'] == df[f'{prefix}choice']).astype(float)
    df.loc[:, f'{prefix}flipped_correct'] = (df['Origin'] == df[f'{prefix}flipped_choice']).astype(float)
    df.loc[:, f'{prefix}avg_correct'] = (df[f'{prefix}correct'] + df[f'{prefix}flipped_correct']) / 2
    return df[f'{prefix}correct'].mean(), df[f'{prefix}flipped_correct'].mean(),df[f'{prefix}avg_correct']. mean()


In [91]:
# File paths
# file_paths = [
#     '/data2/borito1907/sandcastles/distinguisher/results/long_InternLMOracle_GPT4o_unwatermarked_SentenceMutator_news_SimpleDistinguisher.csv',
#     '/data2/borito1907/sandcastles/distinguisher/results/long_InternLMOracle_GPT4o_unwatermarked_SentenceMutator_paris_SimpleDistinguisher.csv',
#     '/data2/borito1907/sandcastles/distinguisher/results/long_InternLMOracle_GPT4o_unwatermarked_SentenceMutator_space_SimpleDistinguisher.csv'.
# ]

dir = '/data2/jasonl/sandcastles/distinguisher/results'

# file_paths = [os.path.join(dir, f) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f))]
# file_paths = [file for file in file_paths if "Simple" in file]


combined_dataframes = combine_files_by_base_name(dir)

# Process each combined DataFrame
all_attacks = []
all_num_attacks = []
for base_name, combined_df in combined_dataframes.items():
    combined_df['domain'] = base_name.split('_')[-1]  # Infer domain from base name
    attacks = separate_attacks(combined_df)
    # attacks = add_entropy_column(attacks)

    num_attacks = [attack[attack['Num'] >= 100] for attack in attacks]

    all_attacks.extend(attacks)
    all_num_attacks.extend(num_attacks)


# Combine original DataFrames for concatenated view (optional)
long = pd.concat(all_attacks, ignore_index=True)

In [92]:
len(long)

900

In [93]:
data = []

for attack in all_num_attacks:    
    correct, flipped_correct, avg_correct = return_correct_flipped_correct(attack)
    d = check_unique_values(attack, ['domain', 'entropy'])

    data.append({
        'domain' : d['domain'],
        'entropy' : d['entropy'],
        'correct': correct,
        'flipped_correct': flipped_correct,
        'avg_correct': avg_correct
    })
    
data_df = pd.DataFrame(data)

In [217]:
from collections import defaultdict
counter = defaultdict(int)
for attack in all_attacks:
    counter[attack['avg_correct'].tail(5).values[0]] += 1
counter.items()

dict_items([(1.0, 159), (0.5, 21)])

In [94]:
grouped_df = (
    data_df.groupby('entropy')
      .agg({
          'correct': 'mean',
          'flipped_correct': 'mean',
          'avg_correct': 'mean'
      })
      .reset_index()
)

print(grouped_df)

   entropy   correct  flipped_correct  avg_correct
0      1.0  0.933333         0.888889     0.911111
1      2.0  0.833333         0.966667     0.900000
2      3.0  0.975000         0.937500     0.956250
3      4.0  1.000000         0.971429     0.985714
4      5.0  1.000000         0.960000     0.980000
5      6.0  0.844444         0.866667     0.855556
6      7.0  0.966667         0.950000     0.958333
7      8.0  1.000000         0.980000     0.990000
8      9.0  0.950000         0.933333     0.941667
9     10.0  1.000000         1.000000     1.000000


In [95]:
grouped_df = (
    data_df.groupby('entropy')
      .agg({
          'correct': 'std',
          'flipped_correct': 'std',
          'avg_correct': 'std'
      })
      .reset_index()
)

print(grouped_df)

   entropy   correct  flipped_correct  avg_correct
0      1.0  0.237635         0.249444     0.160473
1      2.0  0.349892         0.115470     0.175810
2      3.0  0.100000         0.140831     0.081394
3      4.0  0.000000         0.106904     0.053452
4      5.0  0.000000         0.084327     0.042164
5      6.0  0.287427         0.205798     0.154243
6      7.0  0.115470         0.124316     0.079296
7      8.0  0.000000         0.063246     0.031623
8      9.0  0.124316         0.177525     0.099620
9     10.0  0.000000         0.000000     0.000000


In [99]:
# Create a list to store the attacks and their corresponding avg values
attack_avg_list = []

for i, attack in enumerate(all_attacks):
    attack['id'] = i
    correct, flipped, avg = return_correct_flipped_correct(attack)
    attack_avg_list.append((attack, avg))  # Append the attack and its avg value as a tuple

# Sort the list by the avg value (second element of the tuple)
sorted_attacks = sorted(attack_avg_list, key=lambda x: x[1])

# Extract the sorted attacks and their averages separately if needed
sorted_attacks_list = [item[0] for item in sorted_attacks]
sorted_avgs = [item[1] for item in sorted_attacks]

# # Optionally, print the sorted averages for verification
# for attack, avg in sorted_attacks:
#     print(f"Attack: {attack}, Avg: {avg}")


In [102]:
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, IntSlider

# Function to plot cumulative avg progression for a given attack index
def plot_cumulative_avg(attack_index):
    # Get the attack for the given index
    attack = sorted_attacks_list[attack_index]

    avg_values = []
    
    # Iterate over the rows of the DataFrame cumulatively
    for i in range(1, len(attack) + 1):
        # Select the first `i` rows
        subset = attack.iloc[:i]
        
        # Apply `return_correct_flipped_correct` to the subset
        correct, flipped, avg = return_correct_flipped_correct(subset)
        
        # Store the avg value
        avg_values.append(avg)

    # Plot the graph
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(avg_values) + 1), avg_values, marker='o')
    plt.title(f'Cumulative Avg Progression for Attack Index {attack_index} (id={attack["id"].iloc[0]})')
    plt.xlabel('Number of Rows')
    plt.ylabel('Average')
    plt.grid(True)
    plt.show()

# Function to plot rolling avg progression for a given attack index
def plot_rolling_avg(attack_index, window_size=10):
    # Get the attack for the given index
    attack = sorted_attacks_list[attack_index]

    # Calculate rolling average
    rolling_avg = attack['avg_correct'].rolling(window=window_size).mean()

    # Plot the graph
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(rolling_avg) + 1), rolling_avg, marker='o')
    plt.title(f'Rolling Avg Progression for Attack Index {attack_index} (Window Size: {window_size}), (id={attack["id"].iloc[0]})')
    plt.xlabel('Number of Rows')
    plt.ylabel('Rolling Average')
    plt.grid(True)
    plt.show()


# Create an interactive slider to select attack index
def plot_selected(attack_index, plot_type, window_size=10):
    if plot_type == 'cumulative':
        plot_cumulative_avg(attack_index)
    elif plot_type == 'rolling':
        plot_rolling_avg(attack_index, window_size)

interact(
    plot_selected,
    attack_index=IntSlider(min=0, max=len(sorted_attacks_list) - 1, step=1, value=100),
    plot_type={
        'Cumulative Average': 'cumulative',
        'Rolling Average': 'rolling'
    },
    window_size=IntSlider(min=1, max=50, step=1, value=10)
)

interactive(children=(IntSlider(value=100, description='attack_index', max=179), Dropdown(description='plot_ty…

<function __main__.plot_selected(attack_index, plot_type, window_size=10)>

In [168]:
sorted_attacks_list[0][['Origin', 'choice', 'flipped_choice', 'correct', 'flipped_correct', 'avg_correct']].tail(5)

,Origin,choice,flipped_choice,correct,flipped_correct,avg_correct
5,B,A,B,0.0,1.0,0.5
6,B,A,B,0.0,1.0,0.5
7,B,A,B,0.0,1.0,0.5
8,B,A,B,0.0,1.0,0.5
9,B,A,B,0.0,1.0,0.5


In [169]:
from IPython.display import display, HTML

def show(df):
    # Display the DataFrame in a scrollable container
    display(HTML('<div style="max-height: 300px; overflow-y: scroll;">{}</div>'.format(
        df.to_html(index=False)
    )))

In [171]:
show(sorted_attacks_list[0][['Origin', 'prompt', 'P', 'origin_A', 'origin_B']].tail(2))

Origin,prompt,P,origin_A,origin_B
B,Write a 500-word news article summarizing a local teacher strike.,". **Schools Closed After Teachers Strike in Cityville, CA** Teachers took their grievances and demands to the streets today, as they went on strike to take a stand for higher pay, smaller class sizes and most importantly, increased funding. The strike, organized and launched by the California Federation of Teachers, effectively shut down all of the classes at the 25 schools in the city, affecting 30,000 students in the process.\n\nThe picket lines began to form early, as teachers, many of them wearing red, chanted slogans of ""Support Our Cityville Teachers!"" and ""Fair Pay for Fair Work."" Spirits were high, and chants were loud as the group stood unified, showing signs of solidarity among the ranks. The decision to strike was made following weeks of failed negotiations between the union, and representatives of the school board. At the heart of the walkout is a 6% per year increase to salaries, in order for teachers to keep up with a cost of living increase. Additionally, teachers are asking for smaller class sizes, more professional development opportunities and funding to provide individualized education programs, that meets the needs of students. ""The demands placed upon on teachers in this city are out of control, and are simply unsustainable,"" said Michael Brown, president of the union. ""The educators in this district are highly skilled, highly valued professionals, but we have been pushed to our limits. We're here to say that we've had enough!"" Residents of the school district have had a mixed reaction to the walkout. Some parents voiced support, acknowledging the need for the kinds of changes that teachers are outrightly demanding. ""If we want to truly have schools that thrive, and creates an environment of success, where everyone feels comfortable, we have to ensure that teachers are being treated well, and are being well taken care of,"" said David, a parent. Others voiced their own personal concern about student safety, and worried about the impact on the strike on their young ones. ""I get where they're coming from, but I worry about the how this is affecting the children,"" said Karen, mother of a girl in 1st grade. ""This school year is stressful enough for kids, but then to not have school is stressful.""\n\nThe Cityville School Board has voiced regret that this has happened, as they wish the teachers had given them a better an alternative to consider before deciding to undertake such a walkout. ""We are focusing on keeping students safe, and are willing to welcome the teachers back to the classrooms, at any time,"" said John Carter, board president. ""Our priority is getting students back to school as quickly and safely as possible."" Afterschool programs, recreation centers, senior centers and local churches are offering open learning centers to try to keep students occupied, and to help provide them with some form of guidance. Without school, students are facing uncertain days, as they are relying on their parents and guardians to keep them occupied and learning. The situation unfolding in Cityville mirrors broader, national issues in education, as teachers throughout the nation go on strike to demand that public school districts do better. While teachers' strikes may seem radical, their activism highlights the need for teachers, and education reforms, that they feel have been overlooked. As this new week begins, negotiations between representatives of the union and the School of Education Board are facing an uphill battle to come to a conclusion. As it has each day, the situation becomes more dire, both for teachers and for students. The Cityville, CA walkout sends a message to policymakers, and to the community. It provides an important reminder of the importance of funding the future, and the professionals at the forefront of the education system who are the ones that are responsible for creating th

In [172]:
# convert the list of attacks to a single dataframe
first_rows = [attack.iloc[0].to_frame().T for attack in sorted_attacks_list]
attacks_df = pd.concat(first_rows, ignore_index=True)
# add sorted_avgs as a column to the dataframe
attacks_df['avg_correct'] = sorted_avgs
attacks_df[['o_str', 'w_str', 'm_str', 'avg_correct']][:20]

,o_str,w_str,m_str,avg_correct
0,InternLMOracle,distinguisher,SpanMutator,0.5
1,InternLMOracle,distinguisher,SpanMutator,0.5
2,InternLMOracle,distinguisher,SpanMutator,0.5
3,InternLMOracle,distinguisher,SpanMutator,0.6
4,InternLMOracle,distinguisher,SpanMutator,0.6
5,InternLMOracle,distinguisher,SentenceMutator,0.6
6,InternLMOracle,distinguisher,SentenceMutator,0.6
7,InternLMOracle,distinguisher,SentenceMutator,0.6
8,InternLMOracle,distinguisher,SentenceMutator,0.6
9,InternLMOracle,distinguisher,SentenceMutator,0.6


In [181]:
i = 2
origin_A = sorted_attacks_list[i]['origin_A'].values[0]
origin_B = sorted_attacks_list[i]['origin_B'].values[0]
P = sorted_attacks_list[i]['P'].values[-1]

print(f'''Modified text: {P}
___________________________________________________________
Original response A: {origin_A}
___________________________________________________________
Original response B: {origin_B}
___________________________________________________________
Which response does the modified text come from?

Reply with only one character.
''')

Modified text: **Title: Teachers Strike for Higher Pay, Smaller Class Sizes; Community Reacts**

**By: Paula R. Smith** In an unlikely turn of events, one that leaves a substantial community in the balance, Harrisburg school teachers have been on an extended strike, demanding higher pay and smaller class sizes. The strike, which began Monday, leaves Harrisburg classrooms empty, with some students in temporary care elsewhere, and some large-scale demonstrations outside school buildings across the district.

**Teachers’ Demands** The teachers, working under the leadership of the local union, have made clear they want an over-all increase in pay and a reduction in class sizes. They argue that the current pay structure of the district is drastically underfunded and inadequate, as it does not properly cover the costs of education or reflect the needs of the district. "We are here for the students," said Melissa Johnson, an 8th-grade math teacher and a member of the union local. "We are desp

In [ ]:
print(f'''Modified text: {P}
___________________________________________________________
Original response A: {origin_B}
___________________________________________________________
Original response B: {origin_A}
___________________________________________________________
Which response does the modified text come from?

Reply with only one character.
''')

In [182]:
sorted_attacks_list[i].tail(1)

,P,Num,Origin,choice,flipped_choice,prompt,origin_A,origin_B,o_str,w_str,m_str,prompt_type,entropy,attack1_id,attack2_id,domain,correct,flipped_correct,avg_correct,id
49,"**Title: Teachers Strike for Higher Pay, Small...",199,B,A,B,Write a 500-word news article about a local te...,**Local Teacher Strike Demands Higher Pay and ...,**Title: Teachers Strike for Higher Pay and Sm...,InternLMOracle,distinguisher,SpanMutator,test,2,3,5,test,0.0,1.0,0.5,69
